In [1]:
!pip install numpy==1.23.5  
!pip install pandas==1.5.3  
!pip install tensorflow==2.11.0

In [2]:
pip install nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [3]:
# pip freeze > requirements.txt Command to generate requirements.txt file with dependency list

In [4]:
import os
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
import numpy as np
import tensorflow as tf
import tensorflow.lite as tflite

app = FastAPI()

current_dir = os.getcwd()
model_path = os.path.join(current_dir, "activity_recognition_model500.tflite")

# Carica il modello TensorFlow Lite
interpreter = tflite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Ottieni i dettagli degli input e output del modello
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Definizione del modello di input
class TimeSeriesInput(BaseModel):
    data: List[List[float]]

@app.post("/predict/")
async def predict(timeseries: TimeSeriesInput):
    try:
        # Verifica la dimensione del tensore
        if len(timeseries.data) != 500 or any(len(row) != 9 for row in timeseries.data):
            raise HTTPException(status_code=400, detail="La dimensione del tensore deve essere 500x9")
        
        # Converte l'input in un numpy array
        input_array = np.array(timeseries.data, dtype=np.float32)
        input_array = input_array.reshape((1, 500, 9))  # Modifica la forma per il modello
        
        # Prepara l'interprete
        interpreter.set_tensor(input_details[0]['index'], input_array)
        interpreter.invoke()

        # Ottieni il risultato
        output_data = interpreter.get_tensor(output_details[0]['index'])
        predicted_class = np.argmax(output_data, axis=1)
        
        return {"predicted_class": int(predicted_class[0])}
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
        
@app.get("/helloworld/") 
async def hello_world(): 
    return {"message": "Hello, world!"}



In [ ]:
import nest_asyncio
import uvicorn

# Applica il patching di asyncio
nest_asyncio.apply()

if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [29656]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:17279 - "GET /helloworld HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:17279 - "GET /helloworld/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:17295 - "GET /helloworld HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:17295 - "GET /helloworld/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:17635 - "GET /helloworld HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:17635 - "GET /helloworld/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:17691 - "GET /helloworld HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:17691 - "GET /helloworld/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:17704 - "GET /helloworld HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:17704 - "GET /helloworld/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:17733 - "GET /helloworld HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:17733 - "GET /helloworld/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:17741 - "POST /predict/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:52439 - "GET /helloworld HTTP/1.1" 307 Temporary Redirect
INFO:     127.0